## Покращення парсера залежностей

In [1]:
import conllu
import random
from collections import OrderedDict
import string
import gzip

from tokenize_uk import tokenize_words
import pymorphy2
morph = pymorphy2.MorphAnalyzer(lang='uk')

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier 
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib

Парсери залежностей використовують різні алгоритми переходів, типи оракулів, ознаки для передбачення тощо. Для цієї домашньої роботи я спробував три підходи (усі з алгоритмом Arc-Eager):

- парсер зі статичним оракулом - приблизно той, що ми будували на практичному занятті, з додатковими ознаками);
- парсер зі статичним оракулом і промаркованими типами залежностей (labeled parser) - на основі попереднього парсера;
- парсер із динамічним оракулом.

In [2]:
# get train and test data
fname = 'uk_iu-ud-train.conllu.gz'
with gzip.open(fname, 'rb') as f:
    raw_train = f.read().decode()

fname2 = 'uk_iu-ud-test.conllu.gz'
with gzip.open(fname2, 'rb') as f2:
    raw_test = f2.read().decode()

### Парсер №1

In [3]:
class Parser():
    """
    Here we parse dependency tree with known dependencies,
    learn features for classifier, and try to parse trees
    with unknown dependencies given features.
    """
    
    def __init__(self):
        self.ROOT = OrderedDict({'form': 'ROOT', 'id': 0, 'head': -1, 
                                 'lemma': 'ROOT', 'upostag': 'ROOT',
                                 'deprel': 'root'})
        
    def read_raw_conllu(self, raw_text):
        """
        Parse CONLLU text with corresponding library.
        Returns list of parsed sentences.
        """
        trees = conllu.parse(raw_text)
        return trees

    def make_action(self, action, stack, queue, relations):
        """
        Applies action to the stack, the queue, and the relations.
        """
        w1 = stack[-1]
        w2 = queue[0]
        if action == 'SHIFT':
            stack.append(queue.pop(0))
        elif action == 'REDUCE':
            stack.pop()
        elif action == 'LEFT':
            relations.append((w1['id'], w2['id']))
            stack.pop()
        elif action == 'RIGHT':
            relations.append((w2['id'], w1['id']))
            stack.append(queue.pop(0))
        return stack, queue, relations

    def oracle(self, top_stack, top_queue, relations):
        """
        Returns the right action given the state
        of the stack, the queue, and the relations.
        """
        if top_stack and not top_queue:
            return 'REDUCE'
        elif top_queue["head"] == top_stack["id"]:
            return 'RIGHT'
        elif top_stack["head"] == top_queue["id"]:
            return 'LEFT'
        elif (top_stack["id"] in [i[0] for i in relations] and 
             top_queue["head"] < top_stack["id"]):
            return 'REDUCE'
        else:
            return 'SHIFT'

    def apply_actions(self, tree, train=False):
        """
        Produce dependencies for the tree with known dependencies.
        If train=True, also get features and labels in the process.
        """
        stack = [self.ROOT]
        queue = tree[:]
        relations = []
        labels = []
        features = []
        while stack and queue:
            top_stack = stack[-1] if stack else None
            top_queue = queue[0] if queue else None
            action = self.oracle(top_stack, top_queue, relations)
            if train:
                labels.append(action)
                features.append(self.get_features(stack, queue, relations, tree))
            stack, queue, relations = self.make_action(action, stack, queue, relations)
        if train:
            return relations, labels, features
        return relations
    
    def get_morph_feats(self, word):
        """
        Get what is in 'feats' dict of the parsed word.
        """
        if (not 'feats' in word
            or word['feats'] is None):
            return {}
        features = dict()
        for k in word['feats'].keys():
            features[k] = word['feats'][k]
        return features
    
    def get_child_deps(self, wid, relations, tree):
        """
        Get rightmost and leftmost child dependencies.
        """
        children = list(filter(lambda x: x[1] == wid, relations))
        if len(children):
            nchildren = len([d for (d, h) in children])
            lc1 = min(d for (d, h) in children)
            rc1 = max(d for (d, h) in children)
            return tree[lc1], tree[rc1], nchildren
        return None
        
    def get_features(self, stack, queue, relations, tree):
        """
        Create a dictionary of features for each action.
        """
        features = {}
        if stack:
            w = stack[-1]
            features.update({
                'st0form': w['form'],
                'st0lemma': w['lemma'],
                'st0pos': w['upostag'],
                'st0form_pos': w['form']+'_'+w['upostag']
            })
            if self.get_child_deps(w['id'], relations, tree):
                left_pos, right_pos, nchildren = self.get_child_deps(w['id'], relations, tree)
                features.update({
                    'st0leftc_pos': left_pos['upostag'],
                    'st0rightc_pos': right_pos['upostag'],
                    'st0nchildren': nchildren
                })
            features.update(self.get_morph_feats(w))
        if len(stack) > 1:
            w = stack[-2]
            features.update({
                'st1pos': w['upostag'],
                'st1form': w['form'],
                'st1form_pos': w['form'] + '_' + w['upostag'],
                'st0_1pos': stack[-1]['upostag'] + '_' + w['upostag']
            })
        if queue:
            w = queue[0]
            features.update({
                'q0form': w['form'],
                'q0lemma': w['lemma'],
                'q0pos': w['upostag'],
                'q0form_pos': w['form']+'_'+w['upostag']
            })
            if self.get_child_deps(w['id'], relations, tree):
                left_pos, right_pos, nchildren = self.get_child_deps(w['id'], relations, tree)
                features.update({
                    'q0leftc_pos': left_pos['upostag'],
                    'q0rightc_pos': right_pos['upostag'],
                    'q0nchildren': nchildren
                })
            features.update(self.get_morph_feats(w))
        if len(queue) > 1:
            w = queue[1]
            features.update({
                'q1form': w['form'],
                'q1lemma': w['lemma'],
                'q1pos': w['upostag'],
                'q1form_pos': w['form'] + '_' + w['upostag'],
                'q0_1pos': queue[0]['upostag'] + '_' + w['upostag']
            })
        if len(queue) > 2:
            w = queue[2]
            features.update({
                'q2pos': w['upostag'],
                'q2form_pos': w['form'] + '_' + w['upostag'],
                'q012pos': queue[0]['upostag'] + '_' + queue[1]['upostag'] + w['upostag']
            })
        if len(queue) > 3:
            w = queue[3]
            features.update({
                'q3pos': w['upostag']
            })
        if stack and queue:
            w1 = stack[-1]
            w2 = queue[0]
            features.update({
                'distance': abs(w1['id'] - w2['id']),
                'st0f_q0f': w1['form'] + '_' + w2['form'],
                'st0p_q0p': w1['upostag'] + '_' + w2['upostag'],
                'st0f_q0p': w1['form'] + '_' + w2['upostag'],
                'st0p_q0f': w1['upostag'] + '_' + w2['form']
            })
        return features

    def equal_to_gold(self, gold_tree, parsed_rel):
        """
        Compare lists of relations, return True if they're the same.
        """
        gold_rel = self.get_relations(gold_tree)
        parsed_rel = sorted(parsed_rel)
        if gold_rel == parsed_rel:
            return True
        return False
    
    def get_relations(self, tree):
        """
        Return list of relation nodes (child, parent)
        """
        return [(w['id'], w['head']) for w in tree]
       
    def _check_parser(self, raw_text):
        """
        Check if our parser creates the right relations,
        given golden trees from CONLLU raw_text.
        """
        wrong_count = 0
        wrong = []
        gold_trees = self.read_raw_conllu(raw_text)
        parsed_rels = [self.apply_actions(tree) for tree in gold_trees]
        for gold_tree, parsed_rel in zip(gold_trees, parsed_rels):
            if self.equal_to_gold(gold_tree, parsed_rel):
                continue
            else:
                wrong_count += 1
                wrong.append(gold_trees.index(gold_tree))        
        if wrong_count > 0:
            print('There are {0} sentences parsed not quite right'.format(wrong_count))
            print(wrong)
        else:
            print('All trees are good')
            return
        
    def UAS_train(self, raw_text):
        """
        Count Unlabeled Attachment Score for "train set".
        That is, what % of dependencies we got right from the start.
        """
        total = 0
        tp = 0
        gold_trees = self.read_raw_conllu(raw_text)
        for tree in gold_trees:
            golden = set(self.get_relations(tree))
            parsed = set(self.apply_actions(tree))
            total += len(tree)
            tp += len(golden.intersection(parsed))
        return round(tp/total, 3)

    def fit_classifier(self, features, labels, seed=505, clf='logistic', penalty='l2'):
        """
        Fit Logistic Regression given vectorized train features and labels
        """
        if clf == 'logistic':
            clf_model = LogisticRegression(random_state=seed, penalty=penalty)
        elif clf == 'sgd':
            clf_model = SGDClassifier(random_state=seed, penalty=penalty,
                                max_iter=1000, tol=1e-3)
        vec = DictVectorizer()
        pipeline = Pipeline([('vec', vec), ('clf', clf_model)])
        pipeline.fit(features, labels)
        return pipeline
    
    def classify_and_report(self, raw_train, raw_test,
                            penalty='l2', clf='logistic'):
        """
        Get all together
        """
        train_features, train_labels = self.get_features_for_set(raw_train)
        test_features, test_labels = self.get_features_for_set(raw_test)
        pipeline = self.fit_classifier(train_features, train_labels, clf=clf, penalty=penalty)
        predicted = pipeline.predict(test_features)
        return classification_report(test_labels, predicted)
    
    def get_features_for_set(self, conllu_raw):
        """
        Get features and labels for all of the trees in given set.
        """
        set_trees = self.read_raw_conllu(conllu_raw)
        features = []
        labels = []
        for t in set_trees:
            _, labs, feats = self.apply_actions(t, train=True)
            features.extend(feats)
            labels.extend(labs)
        return features, labels
    
    def predict_relations(self, tree, pipeline):
        """
        A function that predict relations given classifier (oracle)
        """
        stack, queue, relations = [self.ROOT], tree[:], []
        while queue or stack:
            if stack and not queue:
                stack.pop()
            else:
                features = self.get_features(stack, queue, relations, tree)
                action = pipeline.predict(features)[0]
                if action == 'SHIFT':
                    stack.append(queue.pop(0))
                elif action == 'REDUCE':
                    stack.pop()
                elif action == 'LEFT':
                    relations.append((stack[-1]["id"], queue[0]["id"]))
                    stack.pop()
                elif action == 'RIGHT':
                    relations.append((queue[0]["id"], stack[-1]["id"]))
                    stack.append(queue.pop(0))
                else:
                    print("Unknown action.")
        return sorted(relations)
    
    def train_and_save_model(self, raw, fname='model.pkl', clf='logistic', penalty='l2', seed=505):
        """
        Save vectorizer and classifier for reuse.
        """
        train_features, train_labels = self.get_features_for_set(raw)
        vec = DictVectorizer()
        if clf == 'sgd':
            clf_model = SGDClassifier(random_state=seed, penalty=penalty,
                                      max_iter=1000, tol=1e-3)
        else:
            clf_model = LogisticRegression(penalty=penalty, random_state=seed)
        pipe = Pipeline([('vec', vec), ('clf', clf_model)])
        pipe.fit(train_features, train_labels)
        joblib.dump(pipe, fname)
    
    @classmethod
    def pipeline_from_file(self, fname='model.pkl'):
        """
        Load trained pipeline/model from file.
        """
        pipeline = joblib.load(fname)
        return pipeline
    
    def UAS_test(self, raw_train, raw_test, clf='logistic', penalty='l2', seed=505):
        """
        Use predictions on test set, compute UAS
        """
        train_features, train_labels = self.get_features_for_set(raw_train)
        pipeline = self.fit_classifier(train_features, train_labels, clf=clf, 
                                       penalty=penalty, seed=seed)
        test_trees = self.read_raw_conllu(raw_test)
        total = 0
        tp = 0
        for tree in test_trees:
            golden = [(node["id"], node["head"]) for node in tree]
            predicted = self.predict_relations(tree, pipeline)
            total += len(tree)
            tp += len(set(golden).intersection(set(predicted)))
        uas = round(tp/total, 3)
        return uas

Використаний у парсері статичний оракул та алгоритм Arc-Eager, придатний тільки для проективних дерев, не дають 100% надійності навіть для тих дерев, для яких усі залежності відомі. Точність (unlabeled attachment score) на тренувальному сеті тільки 97.4%:

In [4]:
p = Parser()
p.UAS_train(raw_train)

0.974

Для покращення результатів на тестовому сеті ([бейзлайн - 0.69](https://github.com/vseloved/prj-nlp/blob/master/lectures/08-dep-parser-uk.ipynb)) я додав більше фіч:
- POS-теги для найлівішої та найправішої дитини в уже передбачених залежностях (для одного слова з верхівки стеку і одного слова з початку черги);
- кількість цих дітей;
- морфологічні ознаки слова із підсловника 'feats';
- поєднання форми слова + POS-тегу для слів з верхівки стеку і початку черги;
- біграми і триграми POS-тегів для слів зі стеку, черги, черги+стеку тощо;
- відстань між верхівкою стеку та початком черги (кількість слів у реченні між ними).

Мінус усіх покращень - значне уповільнення тренування.

In [5]:
p = Parser()
p.UAS_test(raw_train, raw_test)

0.763

Дещо покращує результат використання параметру penalty='l1' для логістичної регресії. 

In [6]:
p.UAS_test(raw_train, raw_test, penalty='l1')

0.768

Натомість використання SGDClassifier (який реалізує SVM) покращення не дає:

In [7]:
p.UAS_test(raw_train, raw_test, clf='sgd')

0.76

Також можна подивитись на іншу метрику - показники precision, recall, F1 для кожної з чотирьох дій алгоритму (бейзлайн - 0.81 для кожного показника в нижньому рядку).

In [8]:
print(p.classify_and_report(raw_train, raw_test, penalty='l1'))

             precision    recall  f1-score   support

       LEFT       0.91      0.93      0.92      7352
     REDUCE       0.81      0.71      0.75      5511
      RIGHT       0.81      0.86      0.83      7182
      SHIFT       0.91      0.91      0.91      7757

avg / total       0.86      0.86      0.86     27802



Великою проблемою є низький recall для REDUCE: класифікатор не завжди вчасно "позбувається" слова зі стеку, і це може призвести до купи помилок.

### Парсер №2

In [9]:
class LabeledParser(Parser):
    """
    Dependency parser using static oracle,
    for labeled dependencies.
    """
    def strip_colon(self, deprel):
        """
        Strip the second part of deprel (after colon).
        """
        if ':' in deprel:
            new = deprel.split(':')[0].strip()
            return new
        else:
            return deprel
        
    def make_action(self, action, stack, queue, relations):
        """
        Applies action to the stack, the queue, and the relations.
        """
        w1 = stack[-1]
        w2 = queue[0]
        if action[0] == 'SHIFT':
            stack.append(queue.pop(0))
        elif action[0] == 'REDUCE':
            stack.pop()
        elif action[0] == 'LEFT':
            relations.append((w1['id'], self.strip_colon(w1['deprel']), w2['id']))
            stack.pop()
        elif action[0] == 'RIGHT':
            relations.append((w2['id'], self.strip_colon(w2['deprel']), w1['id']))
            stack.append(queue.pop(0))
        return stack, queue, relations

    def oracle(self, top_stack, top_queue, relations):
        """
        Returns the right action given the state
        of the stack, the queue, and the relations.
        """
        if top_stack and not top_queue:
            return ('REDUCE', '')
        elif top_queue["head"] == top_stack["id"]:
            return ('RIGHT', self.strip_colon(top_queue["deprel"]))
        elif top_stack["head"] == top_queue["id"]:
            return ('LEFT', self.strip_colon(top_stack["deprel"]))
        elif (top_stack["id"] in [i[0] for i in relations] and 
             top_queue["head"] < top_stack["id"]):
            return ('REDUCE', '')
        else:
            return ('SHIFT', '')

    def get_child_deps(self, wid, relations, tree):
        """
        Get rightmost and leftmost child dependencies.
        """
        children = list(filter(lambda x: x[2] == wid, relations))
        if len(children):
            nchildren = len([x for x in children])
            lc1 = min(d for (d, r, h) in children)
            rc1 = max(d for (d, r, h) in children)
            return tree[lc1], tree[rc1], nchildren
        return None
        
    def get_features(self, stack, queue, relations, tree):
        """
        Create a dictionary of features for each action.
        """
        features = {}
        if stack:
            w = stack[-1]
            features.update({
                'st0form': w['form'],
                'st0lemma': w['lemma'],
                'st0pos': w['upostag'],
                'st0form_pos': w['form']+'_'+w['upostag']
            })
            if self.get_child_deps(w['id'], relations, tree):
                left_pos, right_pos, nchildren = self.get_child_deps(w['id'], relations, tree)
                features.update({
                    'st0leftc_pos': left_pos['upostag'],
                    'st0rightc_pos': right_pos['upostag'],
                    'st0nchildren': nchildren
                })
            features.update(self.get_morph_feats(w))
        if len(stack) > 1:
            w = stack[-2]
            features.update({
                'st1pos': w['upostag'],
                'st1form': w['form'],
                'st1form_pos': w['form'] + '_' + w['upostag'],
                'st0_1pos': stack[-1]['upostag'] + '_' + w['upostag']
            })
        if queue:
            w = queue[0]
            features.update({
                'q0form': w['form'],
                'q0lemma': w['lemma'],
                'q0pos': w['upostag'],
                'q0form_pos': w['form']+'_'+w['upostag']
            })
            if self.get_child_deps(w['id'], relations, tree):
                left_pos, right_pos, nchildren = self.get_child_deps(w['id'], relations, tree)
                features.update({
                    'q0leftc_pos': left_pos['upostag'],
                    'q0rightc_pos': right_pos['upostag'],
                    'q0nchildren': nchildren
                })
            features.update(self.get_morph_feats(w))
        if len(queue) > 1:
            w = queue[1]
            features.update({
                'q1form': w['form'],
                'q1lemma': w['lemma'],
                'q1pos': w['upostag'],
                'q1form_pos': w['form'] + '_' + w['upostag'],
                'q0_1pos': queue[0]['upostag'] + '_' + w['upostag']
            })
        if len(queue) > 2:
            w = queue[2]
            features.update({
                'q2pos': w['upostag'],
                'q2form_pos': w['form'] + '_' + w['upostag'],
                'q012pos': queue[0]['upostag'] + '_' + queue[1]['upostag'] + w['upostag']
            })
        if len(queue) > 3:
            w = queue[3]
            features.update({
                'q3pos': w['upostag']
            })
        if stack and queue:
            w1 = stack[-1]
            w2 = queue[0]
            features.update({
                'distance': abs(w1['id'] - w2['id']),
                'st0f_q0f': w1['form'] + '_' + w2['form'],
                'st0p_q0p': w1['upostag'] + '_' + w2['upostag'],
                'st0f_q0p': w1['form'] + '_' + w2['upostag'],
                'st0p_q0f': w1['upostag'] + '_' + w2['form']
            })
        return features
    
    def get_relations(self, tree):
        """
        Return list of relation nodes (child, rel, parent)
        """
        return [(w['id'], self.strip_colon(w['deprel']), w['head']) for w in tree]
       
    def LAS_train(self, raw_text):
        """
        Count Labeled Attachment Score for "train set".
        That is, what % of dependencies we got right from the start.
        """
        total = 0
        tp = 0
        gold_trees = self.read_raw_conllu(raw_text)
        for tree in gold_trees:
            golden = set(self.get_relations(tree))
            parsed = set(self.apply_actions(tree))
            total += len(tree)
            tp += len(golden.intersection(parsed))
        return round(tp/total, 3)
     
    def get_features_for_set(self, conllu_raw):
        """
        Get features and labels for all of the trees in given set.
        """
        set_trees = self.read_raw_conllu(conllu_raw)
        features = []
        labels = []
        for t in set_trees:
            _, labs, feats = self.apply_actions(t, train=True)
            features.extend(feats)
            labels.extend([l[0]+'_'+l[1] for l in labs])
        return features, labels
    
    def predict_relations(self, tree, pipeline):
        """
        A function that predict relations given classifier (oracle)
        """
        stack, queue, relations = [self.ROOT], tree[:], []
        while queue or stack:
            if stack and not queue:
                stack.pop()
            else:
                features = self.get_features(stack, queue, relations, tree)
                action_tuple = pipeline.predict(features)[0]
                action = action_tuple.split('_')[0].strip()
                pred_rel = action_tuple.split('_')[1].strip()
                if action == 'SHIFT':
                    stack.append(queue.pop(0))
                elif action == 'REDUCE':
                    stack.pop()
                elif action == 'LEFT':
                    relations.append((stack[-1]["id"], pred_rel, queue[0]["id"]))
                    stack.pop()
                elif action == 'RIGHT':
                    relations.append((queue[0]["id"], pred_rel, stack[-1]["id"]))
                    stack.append(queue.pop(0))
                else:
                    print("Unknown action.")
        return sorted(relations)
    
    def LAS_test(self, raw_train, raw_test, seed=505, penalty='l2'):
        """
        Use predictions on test set, compute UAS
        """
        train_features, train_labels = self.get_features_for_set(raw_train)
        pipeline = self.fit_classifier(train_features, train_labels, seed=seed, penalty=penalty)
        test_trees = self.read_raw_conllu(raw_test)
        total = 0
        tp = 0
        for tree in test_trees:
            golden = [(node["id"], self.strip_colon(node["deprel"]), node["head"]) for node in tree]
            predicted = self.predict_relations(tree, pipeline)
            total += len(tree)
            tp += len(set(golden).intersection(set(predicted)))
        las = round(tp/total, 3)
        return las

Складність парсера, який маркує залежності, в тому, що кожне поєднання "дія+залежність" рахується як окремий клас, і деякі з цих класів мають дуже малу кількість випадків. Щоб зменшити кількість класів, я видаляю функцією `strip_colon` "уточнення" після двокрапки для тих залежностей, які мають форму "dep:detailed". І все одно у нас лишилось 56 класів.

In [10]:
lp = LabeledParser()
lp.LAS_test(raw_train, raw_test, penalty='l1')

0.745

Незважаючи на потребу маркувати тип кожної залежності, labeled attachment score на тестовому сеті ненабагато нижчий, ніж UAS (якщо я ніде не помилився).

In [11]:
print(lp.classify_and_report(raw_train, raw_test, penalty='l1'))

                 precision    recall  f1-score   support

     LEFT_advcl       0.65      0.41      0.50        63
    LEFT_advmod       0.88      0.94      0.91       549
      LEFT_amod       0.96      1.00      0.98      1375
       LEFT_aux       0.94      0.89      0.92        19
      LEFT_case       0.99      0.99      0.99      1355
        LEFT_cc       0.94      0.94      0.94       541
     LEFT_ccomp       0.00      0.00      0.00         1
  LEFT_compound       0.83      0.33      0.48        15
       LEFT_cop       0.89      0.97      0.93        65
     LEFT_csubj       0.50      0.12      0.20         8
       LEFT_dep       0.00      0.00      0.00         1
       LEFT_det       0.96      0.99      0.98       433
 LEFT_discourse       0.74      0.79      0.76       134
      LEFT_expl       0.86      0.50      0.63        12
      LEFT_iobj       0.67      0.17      0.27        12
      LEFT_mark       0.95      0.93      0.94       217
      LEFT_nmod       1.00    

/home/igor/Bin/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Таблиця за класами показує, що загалом чим менше випадків має клас, тим гірші для нього результати.

Збережемо натреновані моделі парсерів для використання згодом:

In [12]:
p.train_and_save_model(raw_train, fname='model.pkl', penalty='l1')
lp.train_and_save_model(raw_train, fname='model_labeled.pkl', penalty='l1')

### Парсер №3

Динамічний оракул для парсера залежностей описаний у [статті Голдберга і Нівре] (http://www.aclweb.org/anthology/C12-1059). Парсер із таким оракулом описано в статті "Parsing English in 500 Lines of Python", але я використав [імплементацію](https://github.com/dpressel/arcs-py) github-користувача dpressel. У цій імплементації працює "знайомий" нам алгоритм Arc-Eager із чотирма, а не трьома діями (в інших алгоритмах немає REDUCE). 

Я адаптував код у файлах парсера, внісши невеликі зміни та "переклавши" деякі частини на python 3. Найбільші зміни торкнулись файлу fx.py, який містить feature extractor: я додав такі фічі, як відстань, більше біграм, морфологічні ознаки слова із підсловника 'feats' і так далі.

Цей парсер використовує on-line learning і вимагає декілька ітерацій для тренування: в якості класифікатора діє перцептрон, який модифікує ваги для фіч під час кожної ітерації.

In [13]:
from arcs_py.parse import ArcEagerDepParser, GreedyDepParser, Classifier
from arcs_py import fileio
from arcs_py import fx

Формат представлення речень для цього парсера трохи відрізняється.

In [14]:
def read_conllu(path):
    data = conllu.parse(gzip.open(path, 'rb').read().decode())
    sentences = []
    for sent in data:
        sentences.append([(w['form'], w['upostag'], w['head']-1, w['deprel'], w['lemma'], w['feats'])
                               if not w['head'] == 0 
                               else (w['form'], w['upostag'], len(sent), w['deprel'], w['lemma'], w['feats'])
                               for w in sent])
    return sentences

In [15]:
train_sentences = read_conllu('uk_iu-ud-train.conllu.gz')
test_sentences = read_conllu('uk_iu-ud-test.conllu.gz')

Також парсер не працює з непроективними деревами, тому відповідна функція позбувається близько 300 з 4500 тренувальних речень.

In [16]:
def filter_non_projective(gold):
    gold_proj = []
    for s in gold:
        gold_conf = GreedyDepParser.get_gold_conf(s)
        if GreedyDepParser.non_projective(gold_conf) is False:
            gold_proj.append(s)
        #elif opts.v is True:
        #    print('Skipping non-projective sentence', s)
    return gold_proj
    
print(len(train_sentences))
train_sentences = filter_non_projective(train_sentences)
print(len(train_sentences))

4513
4205


In [17]:
feature_extractor = fx.ex
model = Classifier({}, [0, 1, 2, 3])
parser = ArcEagerDepParser(model, feature_extractor)

for i in range(1, 21):
    correct_iter = 0
    all_iter = 0
    random.shuffle(train_sentences)
    for gold_sent in train_sentences:
        correct_s, all_s = parser.train(gold_sent, i)
        correct_iter += correct_s
        all_iter += all_s
    print('fraction of correct transitions iteration %d: %d/%d = %f' 
          % (i, correct_iter, all_iter, correct_iter/float(all_iter)))
parser.avg_weights()

fraction of correct transitions iteration 1: 119196/133614 = 0.892092
fraction of correct transitions iteration 2: 124675/133614 = 0.933098
fraction of correct transitions iteration 3: 127199/133614 = 0.951989
fraction of correct transitions iteration 4: 129029/133614 = 0.965685
fraction of correct transitions iteration 5: 130106/133614 = 0.973745
fraction of correct transitions iteration 6: 130547/133614 = 0.977046
fraction of correct transitions iteration 7: 131140/133614 = 0.981484
fraction of correct transitions iteration 8: 131588/133614 = 0.984837
fraction of correct transitions iteration 9: 132025/133614 = 0.988108
fraction of correct transitions iteration 10: 132269/133614 = 0.989934
fraction of correct transitions iteration 11: 132487/133614 = 0.991565
fraction of correct transitions iteration 12: 132597/133614 = 0.992389
fraction of correct transitions iteration 13: 132680/133614 = 0.993010
fraction of correct transitions iteration 14: 132766/133614 = 0.993653
fraction of cor

Тепер на тестовому сеті:

In [18]:
test = filter_non_projective(test_sentences)
all_arcs = 0
correct_arcs = 0

for gold_test_sent in test:

    gold_arcs = [(gold_test_sent[i][2], i) for i in range(len(gold_test_sent))]
    arcs = parser.run(gold_test_sent)
    correct_arcs += len(set(gold_arcs) & set(arcs))
    all_arcs += len(set(gold_arcs))

print('UAS {ca}/{aa} = {uas}'.format(ca=correct_arcs, aa=all_arcs, 
                               uas=round(correct_arcs/all_arcs, 3)))

UAS 11150/13879 = 0.803


Тут можна зауважити, що тренувальна і тестова вибірка для цього парсера відрізняються, тому що не включають кількасот непроективних дерев. Але я окремо прогнав перший unlabeled парсер на "відфільтрованих" вибірках, і вийшов UAS 0.77 - мінімальне покращення порівняно з 0.768.

### Використання парсерів на нових даних

Для використання парсера на нових даних, потрібно навчитись діставати з них не тільки POS-теги, але і різні морфологічні властивості - це pymorphy2 більш-менш також може робити.

In [19]:
MAPPING = {"ADJF": "ADJ", "ADJS": "ADJ", "COMP": "ADJ", "PRTF": "ADJ",
           "PRTS": "ADJ", "GRND": "VERB", "NUMR": "NUM", "ADVB": "ADV",
           "NPRO": "PRON", "PRED": "ADV", "PREP": "ADP", "PRCL": "PART",
           "PNCT": "PUNCT", "LATN": "X"}

CONJ_COORD = ["а", "або", "але", "ані", "все", "все-таки", "втім", "ж", "же",
              "зате", "і", "й", "ніже", "однак", "одначе", "прецінь", "проте",
              "та", "так", "також", "усе", "усе-таки", "утім", "чи"]

DET = ['інакший', 'його', 'тамтой', 'чий', 'їх', 'інш.', 'деякий', 'ввесь', 'ваш', 
     'ніякий', 'весь', 'інший', 'чийсь', 'жадний', 'другий', 'кожний', 
     'такий', 'оцей', 'скілька', 'цей', 'жодний', 'все', 'кілька', 'увесь', 
     'кожній', 'те', 'сей', 'ін.', 'отакий', 'котрий', 'усякий', 'самий', 
     'наш', 'усілякий', 'будь-який', 'сам', 'свій', 'всілякий', 'всенький', 'її', 
     'всякий', 'отой', 'небагато', 'який', 'їхній', 'той', 'якийсь', 'ин.', 'котрийсь', 
     'твій', 'мій', 'це', 'яка', 'якась', 'ця']

def normalize_pos(word):
    if not word.tag.POS:
        if word.word in string.punctuation:
            return "PUNCT"
        else:
            return "X"
    if word.tag.POS == "CONJ":
        if word.word in CONJ_COORD:
            return "CCONJ"
        else:
            return "SCONJ"
    if word.normal_form in DET:
        return 'DET'
    else:
        return MAPPING.get(word.tag.POS, word.tag.POS)

def morph_feats(word):
    """
    Get morphological features of the word.
    """
    morph_dict = {
        'Gender': word.tag.gender,
        'Case': word.tag.case,
        'Number': word.tag.number,
        'Animacy': word.tag.animacy,
        'Mood': word.tag.mood,
        'Tense': word.tag.tense,
        'Aspect': word.tag.aspect,
        'Person': word.tag.person,
        'Voice': word.tag.voice
    }
    morph_mapping = {'femn': 'Fem', 'nomn': 'Nom', 'gent': 'Gen', 'datv': 'Dat',
                    'ablt': 'Ins', 'accs': 'Acc', 'loct': 'Loc', 'voct': 'Voc',
                    'indc': 'Ind', 'impr': 'Imp', 'futr': 'Fut', 'impf': 'Imp',
                    '1per': '1', '2per': '2', '3per': '3', 'actv': 'Act',
                    'pssv': 'Pass'}
    feature_dict = OrderedDict()
    for feat, value in morph_dict.items():
        if not value:
            continue
        if value in morph_mapping.keys():
            feature_dict[feat] = morph_mapping[value]
        else:
            feature_dict[feat] = value.title()
    # let's also add some more verb recognition tools
    if 'GRND' in word.tag:
        feature_dict['VerbForm'] = 'Conv'
    elif 'ADJF' in word.tag and 'pssv' in word.tag:
        feature_dict['VerbForm'] = 'Part'
    elif 'infn' in word.tag:
        feature_dict['VerbForm'] = 'Inf'
    elif 'VERB' in word.tag:
        feature_dict['VerbForm'] = 'Fin'
    return feature_dict

def build_tree(sent):
    """
    Build a tree from sent for parsing.
    """
    words = tokenize_words(sent)
    tree = []
    i = 1
    for word in words:
        wparsed = morph.parse(word)[0]
        word_dict = OrderedDict()
        word_dict['id'] = i
        word_dict['form'] = word
        word_dict['lemma'] = wparsed.normal_form
        word_dict['upostag'] = normalize_pos(wparsed)
        if morph_feats(wparsed):
            word_dict['feats'] = morph_feats(wparsed)
        tree.append(word_dict)
        i += 1
    return tree

def build_tree2(sent):
    """
    Build a tree for a parser with dynamic oracle, of a form:
    (word, POS, head, deprel, lemma, feats)
    """
    words = tokenize_words(sent)
    tree = []
    for word in words:
        wparsed = morph.parse(word)[0]
        postag = normalize_pos(wparsed)
        lemma = wparsed.normal_form
        if morph_feats(wparsed):
            feats = morph_feats(wparsed)
        else:
            feats = OrderedDict()
        word_tuple = (word, postag, None, '', lemma, feats)
        tree.append(word_tuple)
    return tree
        
def dep_parse(sent, model):
    """
    Dependepcy-parse a sentence in Ukrainian.
    """
    tree = build_tree(sent)
    p = Parser()
    relations = p.predict_relations(tree, model)
    return tree, relations

def labeled_parse(sent, model):
    """
    Parse a sentence in Ukrainian and label dependencies.
    """
    tree = build_tree(sent)
    lp = LabeledParser()
    relations = lp.predict_relations(tree, model)
    return tree, relations

def dynamic_parse(sent, parser):
    """
    Parse a sentence in Ukrainian with dynamic oracle.
    """
    tree = build_tree2(sent)
    relations = parser.run(tree)
    return tree, sorted(relations, key = lambda x: x[1])
    
def visualize_tree(tree, relations):
    """
    Draw some arrows.
    """
    if len(relations[0]) == 3:
        for rel in relations:
            w1, deprel, w2 = rel
            word1 = tree[w1-1]['form']
            if w2 == 0:
                word2 = 'ROOT'
            else:
                word2 = tree[w2-1]['form']
            line = '{w1} --{deprel}--> {w2}'.format(
                w1=word1, deprel=deprel, w2=word2)
            print(line)
    else:
        for rel in relations:
            w1, w2 = rel[0], rel[1]
            word1 = tree[w1-1]['form']
            if w2 == 0:
                word2 = 'ROOT'
            else:
                word2 = tree[w2-1]['form']
            line = '{w1} ---> {w2}'.format(
                w1=word1, w2=word2)
            print(line)

def visualize_dynamic(tree, relations):
    """
    Draw even more arrows.
    """
    for (w1, w2) in relations:
        child_word = tree[w2][0]
        if w1 >= len(tree):
            parent_word = 'ROOT'
        else:
            parent_word = tree[w1][0]
        line = '{child} ---> {parent}'.format(
            child=child_word, parent=parent_word)
        print(line)

In [20]:
unlab_model = Parser().pipeline_from_file('model.pkl')
lab_model = LabeledParser().pipeline_from_file('model_labeled.pkl')

In [21]:
sentences = ["Літературної англійської мови, такої яка є в інших країнах, де існують спеціальні інституції,\
 що затверджують мовний стандарт, як наприклад Французька академія у Франції, не існує",
 "Запишіть ваші спостереження та результати в окремий файл",
 "Іноді деякі товари буває важко продати через зіпсовану упаковку чи якісь пошкодження,\
 які не критично впливають на їх функції",
 "Дівчина стояла там, де й була, і намагалася привести до ладу скуйовджене волосся,\
 вкрай розлючена тим, що це побачили водії, які чекали на переїзді",
 "Тисячі людей знову вийшли на вулиці Єревана після заклику лідера опозиції продовжити протести,\
 щоб завершити оксамитову революцію",
 "Одного ранку, прокинувшись од неспокійного сну, Грегор Замза побачив, що він обернувся на страхітливу комаху"]

In [22]:
def try_parsers(sent):
    print('Unlabeled parsing:\n')
    tree, rels = dep_parse(sent, unlab_model)
    visualize_tree(tree, rels)
    print('\n=======================\n')
    print('Labeled parsing:\n')
    tree, rels = labeled_parse(sent, lab_model)
    visualize_tree(tree, rels)
    print('\n=======================\n')
    print('Dynamic oracle parsing:\n')
    tree, rels = dynamic_parse(sent, parser)
    visualize_dynamic(tree, rels)
    print('')

In [23]:
try_parsers(sentences[0])

Unlabeled parsing:

Літературної ---> мови
англійської ---> мови
мови ---> є
, ---> є
такої ---> яка
яка ---> є
є ---> ROOT
в ---> країнах
інших ---> країнах
країнах ---> є
, ---> існують
де ---> існують
існують ---> є
спеціальні ---> інституції
інституції ---> існують
, ---> затверджують
що ---> затверджують
затверджують ---> існують
мовний ---> стандарт
стандарт ---> затверджують
, ---> наприклад
як ---> наприклад
Французька ---> академія
академія ---> стандарт
у ---> Франції
Франції ---> академія
, ---> існує
не ---> існує
існує ---> є


Labeled parsing:

Літературної --amod--> мови
англійської --amod--> мови
мови --obl--> існують
, --punct--> є
такої --obj--> є
є --acl--> мови
в --case--> країнах
інших --det--> країнах
країнах --obl--> є
, --punct--> існують
де --discourse--> існують
існують --advcl--> існує
існують --root--> ROOT
спеціальні --amod--> інституції
інституції --obj--> існують
, --punct--> затверджують
що --mark--> затверджують
затверджують --conj--> існують
мовний --a

Це досить складне речення. Тут уже проявилась та проблема, що для деяких слів парсер зі статичним оракулом призначає дві залежності (у цьому випадку лише для деяких ком). Цю проблему можна вирішити, мабуть, змушуючи парсер вибирати найбільш імовірну з двох залежностей.

В усіх парсерів проблеми з рутом: динамічний взагалі обрав іменник, другий парсер має два рути (хоча один з них правильний). Прикметники тут і далі добре промарковано як залежні від іменників, які вони модифікують.

In [24]:
try_parsers(sentences[1])

Unlabeled parsing:

Запишіть ---> ROOT
ваші ---> спостереження
спостереження ---> Запишіть
та ---> результати
результати ---> спостереження
в ---> файл
окремий ---> файл
файл ---> Запишіть


Labeled parsing:

Запишіть --root--> ROOT
ваші --det--> спостереження
спостереження --nsubj--> Запишіть
та --cc--> результати
результати --conj--> Запишіть
в --case--> файл
окремий --amod--> файл
файл --obl--> Запишіть


Dynamic oracle parsing:

Запишіть ---> ROOT
ваші ---> спостереження
спостереження ---> Запишіть
та ---> результати
результати ---> спостереження
в ---> файл
окремий ---> файл
файл ---> спостереження



Це просте речення, яке парсери промаркували схожим чином. На цьому реченні я побачив, що парсери маркують "ваші" як підмет, залежний від дієслова. Винен був pymorphy2, який позначає "ваші" і схожі слова як займенники, хоча в тренувальній вибірці це DET, а займенниками є тільки іменнико-подібні слова (часто підмети). Тому я витягнув із тренувальної вибірки список слів, які потрібно позначати як DET, і це речення стало парситись майже ідеально.

In [25]:
try_parsers(sentences[2])

Unlabeled parsing:

Іноді ---> буває
деякі ---> товари
товари ---> буває
буває ---> ROOT
продати ---> буває
через ---> упаковку
зіпсовану ---> упаковку
упаковку ---> продати
чи ---> якісь
якісь ---> пошкодження
пошкодження ---> упаковку
, ---> впливають
які ---> впливають
не ---> критично
критично ---> впливають
впливають ---> продати
їх ---> на
функції ---> впливають


Labeled parsing:

Іноді --nsubj--> буває
деякі --det--> товари
буває --root--> ROOT
важко --advmod--> буває
важко --advmod--> продати
продати --conj--> буває
через --case--> упаковку
зіпсовану --amod--> упаковку
упаковку --obl--> продати
чи --discourse--> пошкодження
якісь --det--> пошкодження
пошкодження --conj--> упаковку
, --punct--> впливають
які --nsubj--> впливають
не --advmod--> критично
критично --advmod--> впливають
впливають --acl--> пошкодження
на --discourse--> функції
їх --obj--> на
функції --obl--> впливають


Dynamic oracle parsing:

Іноді ---> буває
деякі ---> товари
товари ---> буває
буває ---> ROOT
важ

У цьому реченні перший парсер пропустив слово "важко", другий призначив для цього слова дві залежності, а третій призначив два рути :(

In [26]:
try_parsers(sentences[3])

Unlabeled parsing:

Дівчина ---> стояла
стояла ---> ROOT
стояла ---> була
там ---> була
, ---> там
, ---> була
де ---> була
й ---> була
була ---> ROOT
, ---> намагалася
і ---> намагалася
намагалася ---> була
привести ---> намагалася
до ---> привести
ладу ---> до
скуйовджене ---> волосся
волосся ---> до
, ---> розлючена
вкрай ---> розлючена
розлючена ---> намагалася
тим ---> розлючена
, ---> побачили
що ---> побачили
це ---> побачили
побачили ---> намагалася
водії ---> побачили
, ---> чекали
які ---> чекали
чекали ---> побачили
на ---> переїзді
переїзді ---> намагалася


Labeled parsing:

Дівчина --nsubj--> стояла
стояла --root--> ROOT
там --discourse--> була
, --punct--> була
де --discourse--> була
й --cc--> була
була --conj--> стояла
, --punct--> намагалася
і --cc--> намагалася
намагалася --conj--> була
привести --xcomp--> намагалася
до --nsubj--> привести
ладу --nmod--> до
скуйовджене --amod--> волосся
волосся --conj--> до
, --punct--> розлючена
вкрай --advmod--> розлючена
розлючена 

У перших двох випадках по декілька повторів :( А третій парсер порівняно непогано промаркував усе, крім рутів, яких знову два.

In [27]:
try_parsers(sentences[4])

Unlabeled parsing:

Тисячі ---> вийшли
людей ---> Тисячі
людей ---> вийшли
знову ---> вийшли
вийшли ---> ROOT
на ---> продовжити
Єревана ---> вулиці
після ---> заклику
заклику ---> Єревана
лідера ---> заклику
опозиції ---> лідера
продовжити ---> вийшли
протести ---> продовжити
, ---> вийшли
щоб ---> завершити
завершити ---> вийшли
оксамитову ---> революцію
революцію ---> завершити


Labeled parsing:

Тисячі --obl--> вийшли
людей --nmod--> Тисячі
знову --advmod--> вийшли
вийшли --advcl--> завершити
вийшли --root--> ROOT
на --discourse--> продовжити
Єревана --nmod--> вулиці
після --advmod--> заклику
заклику --nmod--> Єревана
лідера --nmod--> заклику
опозиції --nmod--> лідера
продовжити --conj--> вийшли
протести --nsubj--> продовжити
, --punct--> вийшли
, --punct--> завершити
щоб --discourse--> завершити
оксамитову --amod--> революцію
революцію --obj--> завершити


Dynamic oracle parsing:

Тисячі ---> вийшли
людей ---> Тисячі
знову ---> вийшли
вийшли ---> ROOT
на ---> заклику
вулиці ---> 

Тут чомусь ще знову повтори у перших двох парсерах. Третій парсер цього разу найадекватніший, хоча теж є помітні помилки.

In [28]:
try_parsers(sentences[5])

Unlabeled parsing:

Одного ---> ранку
ранку ---> прокинувшись
, ---> прокинувшись
прокинувшись ---> ROOT
прокинувшись ---> побачив
од ---> прокинувшись
неспокійного ---> сну
сну ---> од
, ---> Грегор
Грегор ---> од
Грегор ---> побачив
Замза ---> Грегор
побачив ---> ROOT
, ---> обернувся
що ---> обернувся
він ---> обернувся
обернувся ---> побачив
страхітливу ---> комаху
комаху ---> обернувся


Labeled parsing:

Одного --obl--> побачив
ранку --nmod--> Одного
, --punct--> прокинувшись
прокинувшись --advcl--> побачив
прокинувшись --nmod--> Одного
од --obj--> прокинувшись
неспокійного --amod--> сну
сну --nmod--> од
, --punct--> Грегор
Грегор --nsubj--> прокинувшись
побачив --root--> ROOT
, --punct--> обернувся
що --mark--> обернувся
він --nsubj--> обернувся
обернувся --ccomp--> побачив
страхітливу --amod--> комаху
комаху --obj--> обернувся


Dynamic oracle parsing:

Одного ---> ранку
ранку ---> ROOT
, ---> прокинувшись
прокинувшись ---> ранку
од ---> прокинувшись
неспокійного ---> сну
сну -

А тут третій парсер найгірший: він промаркував "ранку" як рут, хоча pymorphy2 це слово парсить як іменник (з лемою "ранка", але все ж). Тільки другий парсер побачив рут у слові "побачив", але підмет Грегор призначив слову "прокинувшись".

Загальні висновки:

- українська мова має проблему з непроективними деревами. Можливо, варто пробувати також graph-based алгоритми парсингу;
- біграми, триграми, та інші додаткові фічі дають покращення 6-9 відсоткових пунктів до UAS, і 5 відсоткових пунктів до F1 при класифікації;
- невелика кількість тренувальних даних (всього кілька тисяч речень) ускладнює роботу labeled парсера для рідкісних типів залежностей;
- динамічний оракул дозволяє досягнути до 80% UAS на тестовій вибірці;
- я не встиг вирішити проблему, коли парсер призначає більш ніж один head тому самому слову або дає два рути для одного речення. По ідеї, потрібно заборонити функції predict_relations ставити ліву або праву арку для слова, яке вже має head; але тоді функція повинна обрати іншу дію, а отже класифікатор має видавати для розгляду функції не одну дію, а кілька проранжованих за імовірністю;
- в цілому ж тестування на "реальних" реченнях показує відносно непогані результати в коротких реченнях, але дуже дивні рішення в довших і складніших.